# 인공지능 학습용 데이터 객체 정보 수집 프로그램

## 작성정보

1. 작성일자 : 2021. 10. 24.

1. 작성자 : 한국지능정보사회진흥원 인공지능데이터전략팀 김민준 책임연구원

1. 버전정보 : ver.0.1

## 동작방법

1. AI 허브에서 다운로드 받은 인공지능 학습용 데이터 폴더를 선택한다.

1. 객체명이 기입된 정보를 입력한다.

1. 분석시작 구문을 실행한다.

1. 분석 결과를 확인하고, 엑셀로 출력해본다.

### 사전 정의 부

In [28]:
import os
import json
import pandas as pd

# 객체명을 입력한다.
# 현재 class 라는 이름으로 입력되어 있고, 상황에 따라 따옴표 안에 내용을 변경한다.
# 예) class -> id
class_key = 'category_id'
# 객체 통계 조사용 dict 구조
class_dict = {}

# 인공지능 학습용 데이터 폴더 선택
# ver0.1에서는 nb가 있는 폴더를 기본 폴더라고 생각
data_dir = './'
#data_dir = 'D:/sidewalk polygon/주행 중 건물 촬영 이미지'

# 함수 : 입력경로 내의 모든 하부 디렉토리 내의 json 파일을 조사하여 목록을 돌려줌
# 입력 : 입력경로
# 출력 : JSON 파일 리스트
def search_json(in_dir):
    json_list = []
    
    for (path, dir, files) in os.walk(in_dir):
        for filename in files:
            ext = os.path.splitext(filename)[-1]
            if ext == '.json':
                json_list.append(f'{path}/{filename}')
                
    return json_list

# 함수 : class_dict 내 value가 있으면 횟수증가, 없으면 1로 설정
# 입력 : value, dict
def counting_value(val):
    if val in class_dict:
        class_dict[val] = class_dict[val] + 1
    else:
        class_dict[val] = 1

# 함수 : json objects에서 특정 키 값의 횟수를 측정하여 dict에 기록
# 입력 : json object, key
def extract_key(json):
    if isinstance(json, dict):
        for k, v in json.items():
            if isinstance(v, (dict, list)):
                extract_key(v)
            elif k == class_key:
                counting_value(v)
    elif isinstance(json, list):
        for item in json:
            extract_key(item)

### 데이터 폴더 읽기

In [29]:
json_list = search_json(data_dir)

if not json_list:
    print('JSON 파일이 없습니다. 폴더를 확인하고 다시 실행해주세요')
    
else:
    # json 경로정보 정제
    json_list = [elem.replace('\\', '/') for elem in json_list]

    # json 경로정보 확인용
    '''
    for json_file in json_list:
        print(json_file)    
    '''

### 클래스 통계 조사

In [30]:
# json file list 조사
for json_file in json_list:
    with open(json_file, 'r') as f:
        objects = json.load(f)
        extract_key(objects)
        
print(class_dict)

{2: 46467, 3: 8408, 1: 29670}


### 클래스 통계 내보내기

* 출력파일 : stat.csv

In [32]:
# Pandas의 DataFrame 활용
df = pd.DataFrame(data=class_dict, index=[0])
df = (df.T)
df.to_excel('stat.xlsx')

       0
2  46467
3   8408
1  29670
